<nav><a href="https://superfunsql.com/"> Blog</a> | <a href="https://github.com/Ian-Fogelman"> Github</a> | <a href="https://www.linkedin.com/in/ian-fogelman-9995a747/">LinkedIn</a></nav>

<div class="alert alert-block alert-info">
Before this can work download and install the AWS CLI, this can be done via pip 
<br><b>pip install awscli</b>
<br>
<br>Also go ahead and get boto3 module
<br><b>pip install boto3</b>
<br>
</div>

In [12]:
import boto3
from botocore.exceptions import NoCredentialsError

# S3

## Upload to S3

In [10]:
import os
os.environ["awsAccessKey"] = ""
os.environ["awsSecretKey"] = ""

In [11]:
import os
ACCESS_KEY = str(os.environ['awsAccessKey'])
SECRET_KEY = str(os.environ['awsSecretKey'])

In [13]:
def upload_to_aws(local_file, bucket, s3_file):
    s3 = boto3.client('s3',
                      aws_access_key_id=ACCESS_KEY,
                      aws_secret_access_key=SECRET_KEY)

    try:
        s3.upload_file(local_file, bucket, s3_file)
        print("Upload Successful")
        return True
    except FileNotFoundError:
        print("The file was not found")
        return False
    except NoCredentialsError:
        print("Credentials not available")
        return False
    
uploaded = upload_to_aws('Links.txt', 'bucket', 'Links.txt')

Upload Successful


## Read Data from S3 to dataframe

In [ ]:
import os
import boto3
import pandas as pd
import sys
from io import StringIO # Python 3.x

bucket_name = 'bucket'
object_key = 'XXX.csv'
csv_obj = client.get_object(Bucket=bucket_name, Key=object_key)
body = csv_obj['Body']
csv_string = body.read().decode('utf-8')
df = pd.read_csv(StringIO(csv_string))

### Generate Presigned URL

In [ ]:
import logging
import boto3
from botocore.exceptions import ClientError


def create_presigned_url(bucket_name, object_name, expiration=60):
    """Generate a presigned URL to share an S3 object

    :param bucket_name: string
    :param object_name: string
    :param expiration: Time in seconds for the presigned URL to remain valid
    :return: Presigned URL as string. If error, returns None.
    """

    # Generate a presigned URL for the S3 object
    s3_client = boto3.client('s3')
    try:
        response = s3_client.generate_presigned_url('get_object',
                                                    Params={'Bucket': bucket_name,
                                                            'Key': object_name},
                                                    ExpiresIn=expiration)
    except ClientError as e:
        logging.error(e)
        return None

    # The response contains the presigned URL
    return response

# EC2

## Create EC2 Instance with web server in user data

In [ ]:
import boto3
ec2 = boto3.resource('ec2',aws_access_key_id=ACCESS_KEY,
        aws_secret_access_key=SECRET_KEY)

# create a new EC2 instance
instances = ec2.create_instances(
     ImageId='ami-00b6a8a2bd28daf19',
     MinCount=1,
     MaxCount=1,
     InstanceType='t2.micro',
     KeyName='ec2-keypair'
 )

### Reboot Instance

In [174]:
import boto3
from botocore.exceptions import ClientError

xID = 'i-'

ec2 = boto3.client('ec2',
                   aws_access_key_id=ACCESS_KEY,
                   aws_secret_access_key=SECRET_KEY,
                   region_name='us-east-1')

try:
    ec2.reboot_instances(InstanceIds=[xID], DryRun=True)
except ClientError as e:
    if 'DryRunOperation' not in str(e):
        print("You don't have permission to reboot instances.")
        raise

try:
    response = ec2.reboot_instances(InstanceIds=[xID], DryRun=False)
    print('Success', response)
except ClientError as e:
    print('Error', e)

Success {'ResponseMetadata': {'RequestId': 'd9ee2ef8-71d7-40af-9de3-a3e416559070', 'HTTPStatusCode': 200, 'HTTPHeaders': {'content-type': 'text/xml;charset=UTF-8', 'content-length': '231', 'date': 'Sun, 23 Feb 2020 15:55:08 GMT', 'server': 'AmazonEC2'}, 'RetryAttempts': 0}}


## Create Security Group

In [158]:
client = boto3.client('ec2')
response = client.create_security_group(
    Description='XXXX',
    GroupName='XXXX',
    VpcId='vpc-e754ee9d',
    DryRun=False
)

# Lambda

## Call AWS Lambda Function

In [ ]:
#Demo creating a lambda function from AWS Console

In [15]:
import requests
import json
import pandas as pd

url = """https://fl9kofqei0.execute-api.us-east-1.amazonaws.com/default/mystockadvisor?Ticker=ASD"""
result = requests.get(url, verify=False)
print(result.text)

C:\Users\Ian\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


{"statusCode": 200, "body": {"Ticker": "ASD", "recomendation": "SELL!"}}


# Sage Maker

In [23]:
print(iris.DESCR)

.. _iris_dataset:

Iris plants dataset
--------------------

**Data Set Characteristics:**

    :Number of Instances: 150 (50 in each of three classes)
    :Number of Attributes: 4 numeric, predictive attributes and the class
    :Attribute Information:
        - sepal length in cm
        - sepal width in cm
        - petal length in cm
        - petal width in cm
        - class:
                - Iris-Setosa
                - Iris-Versicolour
                - Iris-Virginica
                
    :Summary Statistics:

    ============== ==== ==== ======= ===== ====================
                    Min  Max   Mean    SD   Class Correlation
    ============== ==== ==== ======= ===== ====================
    sepal length:   4.3  7.9   5.84   0.83    0.7826
    sepal width:    2.0  4.4   3.05   0.43   -0.4194
    petal length:   1.0  6.9   3.76   1.76    0.9490  (high!)
    petal width:    0.1  2.5   1.20   0.76    0.9565  (high!)
    ============== ==== ==== ======= ===== ===========

### Create Sample Model

In [1]:
from sklearn.model_selection import train_test_split
from  sklearn import  datasets
from sklearn import tree
from sklearn.metrics import accuracy_score

iris=datasets.load_iris()
x=iris.data
y=iris.target
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=.5)
classifier=tree.DecisionTreeClassifier()
classifier.fit(x_train,y_train)
predictions=classifier.predict(x_test)

print(accuracy_score(y_test,predictions))

0.9466666666666667


### Persist Model

In [2]:
import pickle,os
model_file_pickle = open(str(os.getcwd() + '\\IrisClassifier.pkl'), 'wb')
pickle.dump(classifier, model_file_pickle)
model_file_pickle.close()

### Example Return From Endpoint

In [3]:
def returnPrediction(i):
    predictvalues = ['Iris-Setosa','Iris-Versicolour','Iris-Virginica']
    return predictvalues[i]

returnPrediction(classifier.predict([[7.6, 3. , 6.6, 2.1]])[0])

'Iris-Virginica'

In [4]:
%%writefile Iris_Classifier.py

from flask import Flask, request
import pandas as pd
import numpy as np
import json
import pickle
import os

app = Flask(__name__)
# Load Model and Scaler Files
with open(str(os.getcwd() + '\\IrisClassifier.pkl'), 'rb') as pickle_file:
    model = pickle.load(pickle_file)

def returnPrediction(i):
    predictvalues = ['Iris-Setosa','Iris-Versicolour','Iris-Virginica']
    return predictvalues[i]
    
@app.route('/api', methods=['POST'])
def Make_Iris_Prediction():
    data = json.dumps(request.get_json(force=True)).replace('[','').replace(']','').split(',')
    y = []
    for x in data:
        y.append(float(x))
    i = model.predict([[y[0], y[1] , y[2], y[3]]])[0]
    #                    sl , sw   , pl  , pw
    return str(returnPrediction(i))
 
if __name__ == '__main__':
    # host flask app at port 10001
    app.run(port=10001, debug=True)

Writing Iris_Classifier.py


### Test The Endpoint

In [6]:
import requests
def make_api_request(data):
    url = 'http://127.0.0.1:10001/api'
    r = requests.post(url,data)
    return r.text

In [7]:
make_api_request("[3,3,3,3]")

'Iris-Virginica'

In [8]:
make_api_request("[4.0,2.9,1.4,2.4]")

'Iris-Setosa'

In [9]:
make_api_request("[1.0,1.9,7.4,0.4]")

'Iris-Virginica'